In [1]:
from generate_krome_module import generate_krome_module

In [2]:
elements = {'C': 2, 'O': 4}
path_krome = '/home/valentin/Documents/github_showoff/krome'

generate_krome_module(elements, path_krome)


KROME network successfully written to '/home/valentin/Documents/github_showoff/krome/chem_net_ramses_rt/C_2_O_4/network_C_2_O_4'
Cleared KROME build directory
******************************
      WELCOME TO KROME
******************************

Reading option -compact
Reading option -photoBins (NBINS=1)
Reading option -useN
Reading from file "/home/valentin/Documents/github_showoff/krome/chem_net_ramses_rt/C_2_O_4/network_C_2_O_4"...
Automatic reactions found, searching in data/database/
copied H.gfe
copied C.gfe
copied O.gfe
checking sinks/sources...
checking recombinations...
done!
************************************************
REMINDER: note that, since you are using photon-based
 options, you need to initialize the machinery from
 your main file! Read the manual for further details.
************************************************
Any key to continue...Enthalpy OK!

ODEs needed: 18
Reactions found: 24
Species found: 14
Photo reactions found:  6
Species list, reactions, and info s

In [ ]:
print([f"{el}_{ion}" for el, ion in elements.items()])
name = "_".join(f"{el}_{n}" for el, n in elements.items())
print(name)


In [ ]:
from chemical_network_builder import ChemicalNetworkBuilder
# from astro import constants as c

In [ ]:
builder = ChemicalNetworkBuilder()
elements = {'C': 2, 'O': 4}
path_test_network = './test_network'

builder.build_krome_network(elements, path_test_network)



In [ ]:
print("RR C++ into C+ (Z=6, N=4):")
print(builder.get_RR_rate(Z=6, N=4))
print()

print("DR C++ into C+ (Z=6, N=4):")
print(builder.get_DR_rate(Z=6, N=4))
print()

print("RR O++++ into O+++ (Z=8, N=4):")
print(builder.get_RR_rate(Z=8, N=4))  # should return "0d0"
print()

print("CI O++ into O+++ (Z=8, N=6):")
print(builder.get_CI_rate(Z=8, N=6))  # should return "0d0"

In [ ]:
def prepare_krome_network(path_to_rates,path_to_network,element,ion_number):
    
    elements_Z = element
    max_ion = ion_number
    
    count=1
    network = ''
    #Recombinations
    network += '@var: T = Tgas \n \n \n#RECOMBINATION, from Badnell website : \n#http://amdpp.phys.strath.ac.uk/tamoc/RR \n#http://amdpp.phys.strath.ac.uk/tamoc/DR \n \n@format:idx,R,R,P,rate \n'
    for i in [1,2]:
        for j in range(i):
            network += str(count) + ',' + element_names[i-1] + '+'*(j+1) + ',E,' + element_names[i-1] + '+'*j
            network += ', 0d0 \n'
            count += 1
    network += '\n'
    k=0
    for i in elements_Z:
        for j in range(min(i,max_ion[k])):
            network += str(count) + ',' + element_names[i-1] + '+'*(j+1) + ',E,' + element_names[i-1] + '+'*j
            if((i==16) or ((i==26)&(j>3))):
                network += ', auto \n'
            else:
                network += ', ' + RR_Badnell(path_to_rates,i,i-j-1) + ' + ' + DR_Badnell(path_to_rates,i,i-j-1) + '\n'
            count += 1
        k+=1
        network += '\n'

    #Collisions
    network += '\n \n#COLLISIONS, from Voronov 1997 \n \n@format:idx,R,R,P,P,P,rate \n'
    for i in [1,2]:
        for j in range(i):
            network += str(count) + ',' + element_names[i-1] + '+'*j + ',E,' + element_names[i-1] + '+'*(j+1)  + ',E,E, '
            network += '0d0 \n'
            count+=1
    k=0
    for i in elements_Z:
        for j in range(min(i,max_ion[k])):
            network += str(count) + ',' + element_names[i-1] + '+'*j + ',E,' + element_names[i-1] + '+'*(j+1)  + ',E,E, '
            if((i==16) or (i==26)):
                network += 'auto \n'
            else:
                network += Col_Voronov(path_to_rates,i,i-j) + '\n'
            count += 1
        k+=1
        network += '\n'

    #Photoionization
    network += '\n \n#Photoionization \n \n@photo_start \n@format:idx,R,P,P,rate \n'
    k=0
    for i in elements_Z:
        for j in range(min(i,max_ion[k])):
            network += str(count) + ',' + element_names[i-1] + '+'*j + ',' + element_names[i-1] + '+'*(j+1) + ',E, auto \n'
            count += 1
        k+=1
    network += '\n@photo_stop \n'

    f=open(path_to_network,'w')
    f.write(network)
    f.close()